In [10]:
from mlflow.tracking import MlflowClient
import pandas as pd
mlflow_path = "/home/davina/Private/repos/CRRT/mlruns"
client = MlflowClient(mlflow_path)
adult_run_id = "ad70333e489c457386c53d079028454b"
peds_run_id = "ed0cfbb4ff654c78b61a2299137e5864"

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

run = client.get_run(adult_run_id)
metrics = run.data.metrics

In [13]:
from os.path import join
experiment_id = client.get_experiment_by_name("static_learning").experiment_id
pd.read_csv(join(mlflow_path, experiment_id, run.info.run_id, "artifacts", "xgb_test__dist_comparison_table.txt"), sep="\t")

,fn_vs_tp fn_vs_tn fp_vs_tn fp_vs_tp
0,dx_CCS_CODE_1 ...
1,dx_CCS_CODE_10 ...
2,dx_CCS_CODE_100 ...
3,dx_CCS_CODE_102 ...
4,dx_CCS_CODE_103 ...
...,...
6383,Surgery in Past Week ...
6384,Num Prev CRRT Treatments ...
6385,liver_pt_indicator ...
6386,heart_pt_indicator ...


In [ ]:
for subgroup in ["", "_heart", "_liver", "_infection"]:
    prefix = f"xgb_test{subgroup}"

    ### Confusion Matrix ###
    confusion_matrix = np.reshape(
        [
            metrics[v] for v in 
            [ f"{prefix}__TP", f"{prefix}__FP", f"{prefix}__FN", f"{prefix}__TN" ]
        ],
        (-1,2)
    )
    # ref: https://stackoverflow.com/a/29648332/1888794
    ax = sns.heatmap(confusion_matrix, annot=True, fmt="g", cmap='Blues', cbar=False)
    subgroup = "_all" if subgroup == "" else subgroup
    ax.set_title('Confusion Matrix' + subgroup.replace("_", ": ")+ '\n');
    ax.set_ylabel('\nPredicted Values')
    ax.set_xlabel('Actual Values ');

    ax.xaxis.set_ticklabels(["Recommend", "Not Recommend"])
    ax.yaxis.set_ticklabels(["Recommend", "Not Recommend"])

    ## Display the visualization of the Confusion Matrix.
    plt.show()

In [9]:
table = {}

for metric_name in ["accuracy", "ap", "auroc", "brier", "precision", "recall"]:
    table[metric_name] = {}
    for subgroup in ["", "_heart", "_liver", "_infection"]:
        prefix = f"xgb_test{subgroup}"
        subgroup = "all" if subgroup == "" else subgroup
        table[metric_name][subgroup.replace("_","")] = metrics[f"{prefix}__{metric_name}"]
pd.DataFrame(table)

,accuracy,ap,auroc,brier,precision,recall
all,0.680054,0.780769,0.743511,0.224491,0.704878,0.710074
heart,0.671159,0.761228,0.725555,0.235113,0.692308,0.712871
liver,0.662953,0.758340,0.721328,0.238856,0.681159,0.719388
infection,0.679842,0.737759,0.710659,0.242272,0.682119,0.757353
